In [ ]:
import json
import glob
import numpy as np
import matplotlib
# matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
import cv2
# from skimage import morphology
from collections import deque
import time
start_time = time.time()

def max_value(inputlist):
    "find max value of list in list"
    return max([sublist[-1] for sublist in inputlist])

def brightnessMean(image):
    "Calculate mean value of the selected image segment"
    h = image.shape[0]
    w = image.shape[1]
    total = 0
    cnt = 0
    for y in range(0, h):
        for x in range(0, w):
            if image[y][x] > 0:
                total += image[y][x]
                cnt += 1
    mean = total / cnt
    return mean

def thresHold(image, threshold):
    "remove background color by filtering pixel lower than mean value"
    h = image.shape[0]
    w = image.shape[1]
    for y in range(0, h):
        for x in range(0, w):
        # threshold the pixel
            if image[y, x] < threshold:
                image[y, x] = 0 
    return image

def imgResize(image, scale_percent):
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    "resize image with inter_area method"
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    return resized

def findEdge(image):
    "main function to extract graph and node features using BFS"
    row, col = image.shape
    direction = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    visited = [[False for i in range(col)] for j in range(row)]
    # area = {}
    # edge = {}
    edge = []
    edge_reduce = {}
    queue = deque()
    for i in range(row):
        for j in range(col):
            if (visited[i][j] == False and image[i][j] != 0):
                queue.append((i, j))
                visited[i][j] = True
                count = 0
                # edge[(i,j)] = []
                edge_reduce[(i,j)] = []
                while(queue):
                    (x, y) = queue.popleft()
                    count += 1
                    for move in direction:
                        newX, newY = x + move[0], y + move[1]
                        if newX < 0 or newX >= row or newY < 0 or newY >= col or image[newX][newY] == 0 or visited[newX][newY]:
                            continue
                        visited[newX][newY] = True
                        queue.append((newX, newY))
                        # edge[(i, j)].append([x*col+y, newX*col+newY])
                        edge.append([x*col+y, newX*col+newY])
                # area[(i, j)] = count
    # maxArea = max(area, key=area.get)
    # maxEdge = edge[maxArea]
    # return maxEdge
    return edge
                        
def mapEdgeFeature(image, edge):
    row, col = image.shape
    node_set = set()
    for i in edge:
        for j in i:
            node_set.add(j)
    node_list = list(node_set)
    node_list.sort()
    cnt = 0
    feature = {}
    for i in node_list:
        feature[str(cnt)] = str(image[i//col][i%col])
        for e in edge:
            for n in e:
                if n == i:
                    e[e.index(n)] = cnt
        cnt += 1
    return edge, feature
def extractNumber(path_string):
    idx = path_string.rfind('/')
    idx2 = path_string.rfind('.png', idx)
    return path_string[idx+1:idx2]

def main(input_path, write_path, save_path):
    """
    Main function to read the images with suffix ".png", extract graphs,
    save as ".json" file
    """
    cell_images = glob.glob(input_path + "*.png")
    cell_images.sort(key = lambda x: [len(x), x])
    scale = 50
    cnt = 1234
    
    for image in cell_images:
        
        # print("1. Process finished --- %s seconds ---" % (time.time() - start_time))
        name = extractNumber(image)
        img = cv2.imread(image)
        """
        Convert RGB image to gray scale, brightness in range of (0~255)
        """
        print(name)
        mg_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        mean = brightnessMean(mg_gray)
        # img_cell = thresHold(mg_gray, mean)
        _, img_cell = cv2.threshold(mg_gray, mean, 255, cv2.THRESH_TOZERO)
        
        """
        reduce image size by rescaling
        """
        # print('Original Dimensions : ',img_cell.shape)
        """
        Uncomment following if need to do img resize, scale = x%
        """
        width = int(img_cell.shape[1] * scale / 100)
        height = int(img_cell.shape[0] * scale / 100)
        dsize = (width, height)
        img_cell = cv2.resize(img_cell, dsize)
        # print('Resized Dimensions : ',img_cell.shape)
        """
        If want to show and save resized image, uncomment following code
        """
        # cv2.imshow("Resized image", resized_img)
        # cv2.imwrite(write_path + name + ".png",resized_img)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        """
        Applied canny edge detection to skeletonize the actin cytoskeleton
        """
        skeleton_image = cv2.Canny(img_cell, 0, 300,apertureSize = 7)
        """
        To remove the noisy spots in the image
        """
        se1 = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
        se2 = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
        mask = cv2.morphologyEx(skeleton_image, cv2.MORPH_CLOSE, se1)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, se2)
        mask = mask / 255
        skeleton_image = skeleton_image * mask
        # cv2.imshow("Resized image", skeleton_image)
        cv2.imwrite(write_path + name + ".png",skeleton_image)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        edge = findEdge(skeleton_image)
        """
        when do mapping between edge and node feature, use grayscale img
        """
        if not edge:
            print("edge = None")
            continue
        else:
            edge, feature = mapEdgeFeature(img_cell, edge)
            features = {}
            features["edges"] = edge
            features["features"] = feature
            with open(save_path + "1_" + str(cnt) + ".json", 'w') as outfile:
                cnt += 1
                json.dump(features, outfile)

if __name__ == "__main__":
    concentration = "0_nm/"
    input_path = "../image_read/"
    save_path = "../image_dataset/"
    write_path = "../image_write/"
    main(input_path+concentration, write_path+concentration, save_path+concentration)



